<a href="https://colab.research.google.com/github/lbsocial/data-analysis-with-generative-ai/blob/main/GraphRAG_Social_Media_Neo4j.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 GraphRAG: Retrieval-Augmented Generation with Neo4j Knowledge Graph

**Ask natural-language questions — get graph-powered answers.**

> 💰 **Note:** This notebook is optimized for **paid Gemini API** with rate-limiting delays (20s between operations). Question counts are balanced for tutorial purposes (2 per step) to manage API costs while demonstrating all key features.

In the previous notebook (`Social_Media_ETL_Neo4j_Python.ipynb`) we built a social media knowledge graph in Neo4j with **Users, Tweets, Hashtags, and Places**. Now we will layer **AI** on top of that graph to create a **GraphRAG** pipeline.

**What is GraphRAG?**

Traditional RAG retrieves relevant text chunks from a vector store. **GraphRAG** goes further — it combines:
1. **Vector Search** — find semantically similar tweets using embeddings.
2. **Graph Traversal** — follow relationships (who posted it, where, which hashtags) to enrich context.
3. **LLM Generation** — pass the enriched context to a language model for a grounded answer.

![GraphRAG Pipeline](https://github.com/lbsocial/data-analysis-with-generative-ai/blob/1660b372f63accb8e55ea4b439924ba764639182/image/Gemini_Generated_Image_vslfdxvslfdxvslf.png?raw=true)

**What we will build:**
| Step | Action |
|------|--------|
| 1 | Install dependencies |
| 2 | Connect to Neo4j & Gemini |
| 3 | Create vector embeddings for every Tweet |
| 4 | Build a Neo4j Vector Index |
| 5 | Implement pure Vector Search retrieval |
| 6 | Implement GraphRAG retrieval (Vector + Graph Traversal) |
| 7 | Build the full RAG pipeline with Gemini |
| 8 | Compare Vector-only vs GraphRAG answers |
| 9 | Interactive GraphRAG query |
| 10 | Advanced — Cypher-Augmented Generation |
| 11 | Geospatial queries & Geo-Augmented GraphRAG |


> **Why no LangChain?** Frameworks like LangChain provide convenient abstractions (`GraphCypherQAChain`, `Neo4jVector`, etc.) that can simplify production code. However, in this tutorial we use the **Neo4j Python driver** and **Google GenAI SDK** directly so you can see exactly how each piece works — embedding, vector search, graph traversal, prompt formatting, and LLM generation. Once you understand these building blocks, adopting a framework becomes much easier.

## 🛠️ Step 1: Install Dependencies

In [ ]:
pip install neo4j google-genai python-dotenv -q

## 🔌 Step 2: Connect to Neo4j & Gemini

We need two connections:
- **Neo4j** — our knowledge graph.
- **Google Gemini** — for generating embeddings and LLM answers.

> **Note:** This notebook uses Colab Secrets or Google Drive .env file for credentials (see options below).

**💡 Credential Setup (2 Options):**

The code below will automatically try these methods **in order** until credentials are found:

### Option 1: Colab Secrets (Recommended) 🔑
1. Click the **🔑 key icon** in the left sidebar (Secrets panel)
2. Add three secrets with these exact names:
   - `NEO4J_URI`
   - `NEO4J_PASSWORD`
   - `GOOGLE_API_KEY`
3. Enable **notebook access** for each secret (toggle switch)
4. Run Step 2 - credentials load automatically

### Option 2: Google Drive .env File ☁️
1. Create a `.env` file with your credentials (see `.env.example`)
2. Upload it to **Google Drive → Colab Notebooks** folder
   - Path: `MyDrive/Colab Notebooks/.env`
3. Run Step 2 - it will mount Drive and load automatically

> **Note:** This notebook is designed for Google Colab. If you want to run it locally with Jupyter, you'll need to adapt the credential loading code.

In [ ]:
from neo4j import GraphDatabase
from google import genai
import os
from pathlib import Path

NEO4J_URI = None
NEO4J_PASSWORD = None
GOOGLE_API_KEY = None

# ── Method 1: Try Colab Secrets first ─────────────────────────────
try:
    from google.colab import userdata
    NEO4J_URI = userdata.get('NEO4J_URI')
    NEO4J_PASSWORD = userdata.get('NEO4J_PASSWORD')
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    if all([NEO4J_URI, NEO4J_PASSWORD, GOOGLE_API_KEY]):
        print("✅ Loaded credentials from Colab Secrets")
except:
    pass

# ── Method 2: Try .env file from Google Drive ─────────────────────
if not all([NEO4J_URI, NEO4J_PASSWORD, GOOGLE_API_KEY]):
    try:
        # Mount Google Drive
        from google.colab import drive
        drive.mount('/content/drive', force_remount=False)
        print("📂 Google Drive mounted")
        
        # Try loading from .env in Colab Notebooks folder
        from dotenv import load_dotenv
        drive_env_paths = [
            Path('/content/drive/MyDrive/Colab Notebooks/.env'),
            Path('/content/drive/MyDrive/.env'),
        ]
        
        for env_path in drive_env_paths:
            if env_path.exists():
                load_dotenv(env_path)
                NEO4J_URI = os.getenv('NEO4J_URI')
                NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
                GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
                if all([NEO4J_URI, NEO4J_PASSWORD, GOOGLE_API_KEY]):
                    print(f"✅ Loaded credentials from Google Drive: {env_path}")
                    break
    except Exception as e:
        print(f"⚠️ Could not load from Google Drive: {e}")

# ── Show credential status ────────────────────────────────────────
print(f"\n🔍 Credential status:")
print(f"  NEO4J_URI: {'✓' if NEO4J_URI else '✗'}")
print(f"  NEO4J_PASSWORD: {'✓' if NEO4J_PASSWORD else '✗'}")
print(f"  GOOGLE_API_KEY: {'✓' if GOOGLE_API_KEY else '✗'}")

# ── Validate credentials ──────────────────────────────────────────
if not all([NEO4J_URI, NEO4J_PASSWORD, GOOGLE_API_KEY]):
    print("\n❌ MISSING CREDENTIALS!")
    print("\n📋 Choose one of these options:")
    print("\n  Option 1: Colab Secrets (Recommended)")
    print("    1. Click the 🔑 key icon in the left sidebar")
    print("    2. Add secrets: NEO4J_URI, NEO4J_PASSWORD, GOOGLE_API_KEY")
    print("    3. Enable notebook access for each")
    print("    4. Re-run this cell")
    print("\n  Option 2: Google Drive .env File")
    print("    1. Create .env file with your credentials")
    print("    2. Upload to: Google Drive → Colab Notebooks/.env")
    print("    3. Re-run this cell (will mount Drive automatically)")
    raise ValueError("Missing credentials! Choose Option 1 or Option 2 above.")

NEO4J_AUTH = ("neo4j", NEO4J_PASSWORD)

# ── Initialize Gemini Client ──────────────────────────────────────
client = genai.Client(api_key=GOOGLE_API_KEY)

# ── Verify Neo4j Connection ───────────────────────────────────────
driver = GraphDatabase.driver(NEO4J_URI, auth=NEO4J_AUTH)
driver.verify_connectivity()
print("✅ Connected to Neo4j")

# ── Verify Gemini Connection ──────────────────────────────────────
test = client.models.get(model="gemini-2.0-flash")
print(f"✅ Connected to Gemini ({test.display_name})")

## 📊 Step 2b: Verify the Graph Data

Let's confirm the graph we built in the ETL notebook is ready.

In [ ]:
# Quick health check — count nodes by label
with driver.session(database="neo4j") as session:
    result = session.run("""
        CALL {
            MATCH (t:Tweet) RETURN 'Tweet' AS label, count(t) AS count
            UNION ALL
            MATCH (u:User)  RETURN 'User'  AS label, count(u) AS count
            UNION ALL
            MATCH (h:Hashtag) RETURN 'Hashtag' AS label, count(h) AS count
            UNION ALL
            MATCH (p:Place) RETURN 'Place' AS label, count(p) AS count
        }
        RETURN label, count
    """)
    print("── Graph Node Summary ──")
    for record in result:
        print(f"  {record['label']:>10s}: {record['count']}")

# Sample one tweet with all its relationships
with driver.session(database="neo4j") as session:
    result = session.run("""
        MATCH (u:User)-[:POSTED]->(t:Tweet)-[:LOCATED_AT]->(p:Place)
        OPTIONAL MATCH (t)-[:TAGGED_WITH]->(h:Hashtag)
        RETURN u.username AS user, t.text AS text,
               p.name AS place, collect(h.name) AS hashtags
        LIMIT 3
    """)
    print("\n── Sample Tweets ──")
    for record in result:
        print(f"  @{record['user']} from {record['place']}")
        print(f"    \"{record['text'][:80]}...\"")
        print(f"    Tags: {record['hashtags']}\n")

---
## 🧬 Step 3: Generate Vector Embeddings for Tweets

We use Gemini's `gemini-embedding-001` model to convert every tweet's text into a **3072-dimensional vector**. This vector captures the *semantic meaning* of the text.

We then write each embedding back to the `Tweet` node as a property called `embedding`.

In [ ]:
EMBEDDING_MODEL = "models/gemini-embedding-001"
EMBEDDING_DIM = 3072


def get_embeddings(texts: list[str]) -> list[list[float]]:
    """Get Gemini embeddings for a batch of texts."""
    response = client.models.embed_content(
        model=EMBEDDING_MODEL,
        contents=texts,
    )
    return [emb.values for emb in response.embeddings]

In [ ]:
# ── Check if embeddings already exist ──────────────────────────────
with driver.session(database="neo4j") as session:
    result = session.run("""
        MATCH (t:Tweet)
        RETURN count(t) AS total_tweets,
               count(t.embedding) AS tweets_with_embeddings
    """)
    record = result.single()
    total_tweets = record["total_tweets"]
    tweets_with_embeddings = record["tweets_with_embeddings"]

print(f"📊 Tweet embedding status:")
print(f"   Total tweets: {total_tweets}")
print(f"   Tweets with embeddings: {tweets_with_embeddings}")

# ── Only embed if needed ───────────────────────────────────────────
if tweets_with_embeddings == total_tweets and total_tweets > 0:
    print("\n✅ All tweets already have embeddings! Skipping embedding step.")
    print("   💡 This saves significant API quota!")
else:
    print(f"\n⏳ Need to embed {total_tweets - tweets_with_embeddings} tweets...")

    # Fetch tweets that need embeddings
    with driver.session(database="neo4j") as session:
        result = session.run("""
            MATCH (t:Tweet)
            WHERE t.embedding IS NULL
            RETURN t.id AS id, t.text AS text
        """)
        tweets = [(record["id"], record["text"]) for record in result]

    if len(tweets) == 0:
        print("✅ No tweets need embedding!")
    else:
        print(f"📄 Embedding {len(tweets)} tweets...")

        # ── Batch embed (20 tweets at a time) ─────────────────────
        BATCH_SIZE = 20
        for i in range(0, len(tweets), BATCH_SIZE):
            batch = tweets[i : i + BATCH_SIZE]
            ids = [t[0] for t in batch]
            texts = [t[1] for t in batch]

            embeddings = get_embeddings(texts)

            # Write embeddings back to Neo4j
            with driver.session(database="neo4j") as session:
                session.run(
                    """
                    UNWIND $data AS row
                    MATCH (t:Tweet {id: row.id})
                    SET t.embedding = row.embedding
                    """,
                    data=[{"id": id_, "embedding": emb} for id_, emb in zip(ids, embeddings)],
                )

            print(f"  ✅ Embedded batch {i // BATCH_SIZE + 1}/{(len(tweets) - 1) // BATCH_SIZE + 1}")

        print("\n🎉 All tweet embeddings stored in Neo4j!")

---
## 🗂️ Step 4: Create a Neo4j Vector Index

A **vector index** lets Neo4j perform fast approximate-nearest-neighbor (ANN) search on the embedding property. This is the foundation of our retrieval step.

In [ ]:
INDEX_NAME = "tweet_embeddings"

with driver.session(database="neo4j") as session:
    # Ensure the index is dropped idempotently
    print(f"Attempting to drop existing index '{INDEX_NAME}' if it exists...")
    session.run(f"DROP INDEX {INDEX_NAME} IF EXISTS")
    print(f"✅ Index '{INDEX_NAME}' dropped (if it existed).")

    # Create the vector index with the correct dimensions
    session.run(f"""
        CREATE VECTOR INDEX {INDEX_NAME}
        FOR (t:Tweet)
        ON (t.embedding)
        OPTIONS {{
            indexConfig: {{
                `vector.dimensions`: {EMBEDDING_DIM},
                `vector.similarity_function`: 'cosine'
            }}
        }}
    """)
    print(f"✅ Vector index '{INDEX_NAME}' created with {EMBEDDING_DIM} dimensions.")

# Verify index details, specifically dimensions
with driver.session(database="neo4j") as session:
    # Run SHOW VECTOR INDEXES and then filter in Python
    result = session.run("SHOW VECTOR INDEXES")
    found_index = False
    for record in result:
        if record['name'] == INDEX_NAME:
            found_index = True
            print(f"  Index: {record['name']}")
            print(f"  State: {record['state']}")
            # Extract dimensions from the 'properties' map
            if 'properties' in record and 'indexConfig' in record['properties']:
                config = record['properties']['indexConfig']
                dimensions = config.get('vector.dimensions')
                if dimensions:
                    print(f"  Actual Dimensions: {dimensions}")
                else:
                    print("  Dimensions not found in index properties.")
            else:
                print("  Index configuration properties not found.")
            break
    if not found_index:
        print(f"❌ Index '{INDEX_NAME}' not found after creation.")

---
## 🔍 Step 5: Pure Vector Search (Baseline)

First, let's implement a plain **vector similarity search**. We embed the user's question, then find the `k` most similar tweets. This is what a traditional RAG system does — **no graph knowledge** is used.

In [ ]:
def vector_search(question: str, k: int = 5) -> list[dict]:
    """
    Plain vector search — returns the top-k most similar tweets.
    No graph traversal.
    """
    q_embedding = get_embeddings([question])[0]

    cypher = """
        CALL db.index.vector.queryNodes($index, $k, $embedding)
        YIELD node AS tweet, score
        RETURN tweet.id    AS id,
               tweet.text  AS text,
               tweet.likes AS likes,
               score
        ORDER BY score DESC
    """
    with driver.session(database="neo4j") as session:
        result = session.run(cypher, index=INDEX_NAME, k=k, embedding=q_embedding)
        return [dict(record) for record in result]


# ── Test it ───────────────────────────────────────────────────────
question = "What are people saying about graph databases?"
results = vector_search(question)

print(f"🔎 Vector Search Results for: '{question}'\n")
for i, r in enumerate(results, 1):
    print(f"  {i}. [score={r['score']:.4f}] {r['text'][:90]}")

---
## 🕸️ Step 6: GraphRAG Retrieval (Vector + Graph Traversal)

This is the key innovation. After finding similar tweets via vector search, we **traverse the graph** to collect rich context:

| Traversal | What we get |
|-----------|-------------|
| `Tweet ← POSTED ← User` | Who wrote it? How many followers? |
| `Tweet → LOCATED_AT → Place` | Where was it posted? |
| `Tweet → TAGGED_WITH → Hashtag` | What topics is it about? |
| `User → POSTED → OtherTweets` | What else has this user said? (context expansion) |
| `Hashtag ← TAGGED_WITH ← OtherTweets` | What else is tagged with the same topics? |

This gives the LLM **much richer context** than bare text chunks.

In [ ]:
def graph_rag_search(question: str, k: int = 5) -> list[dict]:
    """
    GraphRAG retrieval — vector search + graph traversal.
    Returns enriched context for each matching tweet.
    """
    q_embedding = get_embeddings([question])[0]

    cypher = """
        // ── 1. Vector Search: find semantically similar tweets ──
        CALL db.index.vector.queryNodes($index, $k, $embedding)
        YIELD node AS tweet, score

        // ── 2. Graph Traversal: enrich with relationships ──
        // Get the author
        MATCH (author:User)-[:POSTED]->(tweet)

        // Get the location
        OPTIONAL MATCH (tweet)-[:LOCATED_AT]->(place:Place)

        // Get all hashtags on this tweet
        OPTIONAL MATCH (tweet)-[:TAGGED_WITH]->(hashtag:Hashtag)

        // ── 3. Context Expansion: other tweets by same author ──
        OPTIONAL MATCH (author)-[:POSTED]->(other_tweet:Tweet)
        WHERE other_tweet.id <> tweet.id

        // ── 4. Context Expansion: co-occurring tweets via hashtags ──
        OPTIONAL MATCH (hashtag)<-[:TAGGED_WITH]-(related_tweet:Tweet)
        WHERE related_tweet.id <> tweet.id

        RETURN tweet.id                        AS tweet_id,
               tweet.text                      AS text,
               tweet.likes                     AS likes,
               tweet.retweets                  AS retweets,
               score,

               // Author context
               author.username                 AS author,
               author.followers                AS author_followers,

               // Location context
               place.name                      AS location,
               place.country                   AS country,

               // Hashtag context
               collect(DISTINCT hashtag.name)   AS hashtags,

               // Expanded context
               collect(DISTINCT other_tweet.text)[0..3]   AS author_other_tweets,
               collect(DISTINCT related_tweet.text)[0..3] AS related_tweets_via_hashtag

        ORDER BY score DESC
    """
    with driver.session(database="neo4j") as session:
        result = session.run(cypher, index=INDEX_NAME, k=k, embedding=q_embedding)
        return [dict(record) for record in result]


# ── Test it ───────────────────────────────────────────────────────
question = "What are people saying about graph databases?"
results = graph_rag_search(question)

print(f"🕸️ GraphRAG Results for: '{question}'\n")
for i, r in enumerate(results, 1):
    print(f"  {i}. [score={r['score']:.4f}] @{r['author']} ({r['author_followers']} followers)")
    print(f"     📍 {r['location']}, {r['country']}")
    print(f"     💬 \"{r['text'][:80]}...\"")
    print(f"     🏷️  Tags: {r['hashtags']}")
    if r['related_tweets_via_hashtag']:
        print(f"     🔗 Related: \"{r['related_tweets_via_hashtag'][0][:60]}...\"")
    print()

---
## 🤖 Step 7: Build the Full RAG Pipeline with Gemini

Now we connect everything: **retrieve** enriched context from the graph, **format** it into a prompt, and **generate** an answer with Gemini.

We build two pipelines side-by-side:
- `rag_answer()` — plain vector retrieval + Gemini
- `graph_rag_answer()` — GraphRAG retrieval + Gemini

In [ ]:
LLM_MODEL = "gemini-2.0-flash"

SYSTEM_PROMPT = """You are a social media analyst assistant.
Answer the user's question based ONLY on the retrieved context below.
If the context doesn't contain enough information, say so.
Always cite specific tweets, users, or locations when relevant.
Be concise but thorough."""


def format_vector_context(results: list[dict]) -> str:
    """Format plain vector search results into a text block."""
    lines = []
    for i, r in enumerate(results, 1):
        lines.append(f"Tweet {i} (similarity={r['score']:.3f}, likes={r['likes']}):\n  \"{r['text']}\"")
    return "\n\n".join(lines)


def format_graph_context(results: list[dict]) -> str:
    """Format GraphRAG results into a rich context block."""
    lines = []
    for i, r in enumerate(results, 1):
        block = f"""Tweet {i} (similarity={r['score']:.3f}):
  Text: "{r['text']}"
  Author: @{r['author']} ({r['author_followers']} followers)
  Location: {r['location']}, {r['country']}
  Engagement: {r['likes']} likes, {r['retweets']} retweets
  Hashtags: {', '.join(r['hashtags'])}"""
        if r.get('author_other_tweets'):
            block += f"\n  Other tweets by @{r['author']}:"
            for t in r['author_other_tweets']:
                block += f"\n    - \"{t[:80]}...\""
        if r.get('related_tweets_via_hashtag'):
            block += f"\n  Related tweets (same hashtags):"
            for t in r['related_tweets_via_hashtag']:
                block += f"\n    - \"{t[:80]}...\""
        lines.append(block)
    return "\n\n".join(lines)


def rag_answer(question: str, k: int = 5) -> str:
    """Traditional RAG: vector search + LLM."""
    results = vector_search(question, k)
    context = format_vector_context(results)

    response = client.models.generate_content(
        model=LLM_MODEL,
        contents=f"Context:\n{context}\n\nQuestion: {question}",
        config=genai.types.GenerateContentConfig(
            system_instruction=SYSTEM_PROMPT,
            temperature=0.2,
        ),
    )
    return response.text


def graph_rag_answer(question: str, k: int = 5) -> str:
    """GraphRAG: vector search + graph traversal + LLM."""
    results = graph_rag_search(question, k)
    context = format_graph_context(results)

    response = client.models.generate_content(
        model=LLM_MODEL,
        contents=f"Context:\n{context}\n\nQuestion: {question}",
        config=genai.types.GenerateContentConfig(
            system_instruction=SYSTEM_PROMPT,
            temperature=0.2,
        ),
    )
    return response.text


print("✅ RAG pipelines ready.")

---
## ⚖️ Step 8: Compare Vector-Only RAG vs GraphRAG

Let's ask the same questions using both approaches and compare the quality of answers.

**What to observe:**
- **Traditional RAG**: Returns text chunks based solely on semantic similarity
- **GraphRAG**: Returns richer context including author info, locations, hashtags, and related content

**Expected difference:** GraphRAG answers should be more detailed and cite specific users, locations, and engagement metrics because it leverages graph relationships, not just text similarity.

**Tutorial Note:** We run 2 comparison questions. Each makes 2 API calls (one for traditional RAG, one for GraphRAG).

In [ ]:
import time

comparison_questions = [
    "What are people saying about graph databases and who are the most active users talking about them?",
    "Which cities generate the most discussion about AI and cloud computing?",
]

for question in comparison_questions:
    print("=" * 80)
    print(f"❓ Question: {question}")
    print("=" * 80)

    print("\n📄 ── TRADITIONAL RAG (Vector Only) ──")
    try:
        print(rag_answer(question))
    except Exception as e:
        print(f"⚠️ Error during Vector RAG: {e}")

    print("\n🕸️ ── GRAPHRAG (Vector + Graph Traversal) ──")
    try:
        print(graph_rag_answer(question))
    except Exception as e:
        print(f"⚠️ Error during GraphRAG: {e}")

    print()  # Blank line between questions

---
## 🧪 Step 9: Interactive GraphRAG Query

Now you can test the system with your own questions! 

**Try modifying the question below to explore:**
- Topics in specific cities: *"What are people discussing in New York?"*
- Technology trends: *"What do people say about blockchain?"*
- User-specific queries: *"What topics does @user123 tweet about?"*

The GraphRAG pipeline will automatically retrieve related tweets, users, and hashtags to provide contextual answers.

In [ ]:
# ── Change this question to anything you want ─────────────────────
your_question = "What topics are trending in San Francisco?"

print(f"❓ {your_question}\n")
print(graph_rag_answer(your_question))

---
## 🔬 Step 10: Advanced — Cypher-Augmented Generation

For structured analytical questions ("How many tweets per city?"), we can let the LLM **generate Cypher queries** directly. This combines the power of graph queries with natural language.

**How it works:**
1. **Generate**: LLM creates a Cypher query based on user's question and graph schema
2. **Execute**: Run the generated query against Neo4j
3. **Summarize**: LLM formats the raw results into a human-readable answer

**Benefits:**
- Handles aggregations, counts, and complex analytical queries
- No need to manually write Cypher for every question
- LLM ensures query matches the actual schema

**Tutorial Note:** We demonstrate 2 analytical questions. In production, you can add validation to check generated queries before execution.

In [ ]:
CYPHER_SYSTEM_PROMPT = """You are a Neo4j Cypher expert. Given the user's question, generate a Cypher query to answer it.

The graph schema is:
- (:User {id, username, name, followers, following, tweet_count})
    -[:POSTED]->(:Tweet {id, text, created_at, likes, retweets, replies, location, embedding})
- (:Tweet)-[:LOCATED_AT]->(:Place {name, country, location})
- (:Tweet)-[:TAGGED_WITH]->(:Hashtag {name})

Rules:
- Return ONLY raw Cypher (no markdown, no explanation, no code fences).
- Use LIMIT 10 unless the user asks for more.
- Use meaningful aliases in RETURN clauses.
"""


def cypher_rag_answer(question: str) -> str:
    """Let the LLM generate a Cypher query, run it, then summarize results."""

    # Step 1: Generate Cypher
    cypher_response = client.models.generate_content(
        model=LLM_MODEL,
        contents=question,
        config=genai.types.GenerateContentConfig(
            system_instruction=CYPHER_SYSTEM_PROMPT,
            temperature=0.0,
        ),
    )
    cypher_query = cypher_response.text.strip()
    
    # Strip markdown code fences if present
    if cypher_query.startswith('```'):
        # Remove ```cypher or ``` at start and ``` at end
        lines = cypher_query.split('\n')
        if lines[0].startswith('```'):
            lines = lines[1:]  # Remove first line with ```cypher
        if lines and lines[-1].strip() == '```':
            lines = lines[:-1]  # Remove last line with ```
        cypher_query = '\n'.join(lines).strip()
    
    print(f"🔧 Generated Cypher:\n{cypher_query}\n")

    # Step 2: Execute the Cypher query
    try:
        with driver.session(database="neo4j") as session:
            result = session.run(cypher_query)
            records = [dict(record) for record in result]
    except Exception as e:
        return f"❌ Cypher execution error: {e}"

    if not records:
        return "No results found."

    # Step 3: Summarize with LLM
    import json
    data_str = json.dumps(records, indent=2, default=str)

    summary_response = client.models.generate_content(
        model=LLM_MODEL,
        contents=f"Question: {question}\n\nQuery results:\n{data_str}",
        config=genai.types.GenerateContentConfig(
            system_instruction="Summarize the following database query results in a clear, human-readable way. Use bullet points or a table if appropriate.",
            temperature=0.2,
        ),
    )
    return summary_response.text


print("✅ Cypher-Augmented Generation ready.")

In [ ]:
# ── Analytical questions that benefit from Cypher ──────────────────
analytical_questions = [
    "Which user has the most followers and what do they tweet about?",
    "How many tweets were posted from each city?",
]

for q in analytical_questions:
    print("=" * 80)
    print(f"❓ {q}")
    print("=" * 80)
    
    try:
        print(cypher_rag_answer(q))
    except Exception as e:
        print(f"⚠️ Error: {e}")
    
    # Delay to avoid rate limits (each question = 2 API calls)
    if analytical_questions.index(q) < len(analytical_questions) - 1:
        print("\n⏳ Waiting 20 seconds to avoid rate limits...")
        time.sleep(20)
    
    print()

---
## 🌍 Step 11: Geospatial Queries — Find Tweets Near a Location

Every Tweet and Place node in our graph has a `point()` property storing longitude/latitude. Neo4j's built-in `point.distance()` function lets us find tweets **within a radius** of any coordinate — no external GIS tools needed.

**Use cases:**
- "What are people tweeting about near Times Square?"
- "Which users are active within 50 km of Tokyo?"
- "What topics trend in a geographic cluster?"

In [ ]:
def find_tweets_near(city_name: str, radius_km: int = 50, limit: int = 10) -> list[dict]:
    """
    Find tweets posted within `radius_km` of a city center.
    Uses Neo4j's point.distance() for server-side geospatial filtering.
    """
    cypher = """
        // Get the city center point
        MATCH (p:Place {name: $city})
        WITH p.location AS center

        // Find tweets within radius
        MATCH (u:User)-[:POSTED]->(t:Tweet)-[:LOCATED_AT]->(place:Place)
        WHERE point.distance(t.location, center) < $radius_m
        OPTIONAL MATCH (t)-[:TAGGED_WITH]->(h:Hashtag)

        RETURN t.text                          AS text,
               u.username                      AS author,
               place.name                      AS place,
               collect(DISTINCT h.name)        AS hashtags,
               t.likes                         AS likes,
               round(point.distance(t.location, center) / 1000.0, 2) AS distance_km
        ORDER BY distance_km ASC
        LIMIT $limit
    """
    with driver.session(database="neo4j") as session:
        result = session.run(cypher, city=city_name, radius_m=radius_km * 1000, limit=limit)
        return [dict(record) for record in result]


# ── Demo: find tweets near multiple cities ─────────────────────────
geo_demos = [
    ("San Francisco, CA", 50),
    ("London, GB", 30),
]

for city, radius in geo_demos:
    results = find_tweets_near(city, radius_km=radius)
    print(f"📍 Tweets near {city} (within {radius} km):\n")
    for i, r in enumerate(results, 1):
        print(f"  {i}. @{r['author']} — {r['distance_km']} km away")
        print(f"     \"{r['text'][:80]}...\"")
        print(f"     🏷️ {r['hashtags']}  ❤️ {r['likes']} likes\n")
    print()
    print("-" * 60)

### 🗺️ Which cities are nearest to each other?

We can also compute **inter-city distances** directly in the graph to see how our Place nodes relate geographically.

In [ ]:
# ── Compute pairwise distances between all cities ──────────────────
with driver.session(database="neo4j") as session:
    result = session.run("""
        MATCH (a:Place), (b:Place)
        WHERE a.name < b.name  // avoid duplicates
        RETURN a.name AS city_a,
               b.name AS city_b,
               round(point.distance(a.location, b.location) / 1000.0, 0) AS distance_km
        ORDER BY distance_km ASC
    """)
    print("── City-to-City Distances ──\n")
    for record in result:
        print(f"  {record['city_a']:>20s}  ↔  {record['city_b']:<20s}  {record['distance_km']:>8.0f} km")

### 🧠 Geo-Augmented GraphRAG

Combine **geospatial filtering + vector search + graph traversal** into a single retrieval function. This answers location-specific questions like:
> *"What are people near London saying about AI?"*

**How it works:**
1. **Geo-filter**: Find tweets within a radius of a city using `point.distance()`
2. **Vector search**: Among geo-filtered tweets, find semantically similar ones
3. **Graph traversal**: Enrich with author, hashtags, and related content
4. **LLM generation**: Synthesize findings into a natural language answer

**Tutorial Note:** We demonstrate 2 geo-scoped questions across different cities to show regional differences in discussions.

In [ ]:
def geo_graph_rag_search(question: str, city_name: str, radius_km: int = 100, k: int = 5) -> list[dict]:
    """
    Geo-filtered GraphRAG: vector search restricted to tweets
    within `radius_km` of a city, then enriched via graph traversal.
    """
    q_embedding = get_embeddings([question])[0]

    cypher = """
        // Get city center
        MATCH (p:Place {name: $city})
        WITH p.location AS center

        // Vector search — find semantically similar tweets
        CALL db.index.vector.queryNodes($index, $k_broad, $embedding)
        YIELD node AS tweet, score

        // Geo filter — keep only tweets within radius
        WHERE point.distance(tweet.location, center) < $radius_m

        // Graph traversal — enrich
        MATCH (author:User)-[:POSTED]->(tweet)
        OPTIONAL MATCH (tweet)-[:LOCATED_AT]->(place:Place)
        OPTIONAL MATCH (tweet)-[:TAGGED_WITH]->(hashtag:Hashtag)

        RETURN tweet.text                       AS text,
               score,
               author.username                  AS author,
               author.followers                 AS author_followers,
               place.name                       AS location,
               place.country                    AS country,
               collect(DISTINCT hashtag.name)   AS hashtags,
               tweet.likes                      AS likes,
               tweet.retweets                   AS retweets,
               round(point.distance(tweet.location, center) / 1000.0, 2) AS distance_km
        ORDER BY score DESC
        LIMIT $k
    """
    with driver.session(database="neo4j") as session:
        # Search a broader pool then geo-filter down
        result = session.run(
            cypher,
            city=city_name,
            radius_m=radius_km * 1000,
            index=INDEX_NAME,
            k_broad=k * 10,  # cast a wider net for vector search
            k=k,
            embedding=q_embedding,
        )
        return [dict(record) for record in result]


def geo_graph_rag_answer(question: str, city_name: str, radius_km: int = 100, k: int = 5) -> str:
    """Geo-filtered GraphRAG + Gemini LLM."""
    results = geo_graph_rag_search(question, city_name, radius_km, k)
    if not results:
        return f"No tweets found near {city_name} matching your question."

    context_lines = []
    for i, r in enumerate(results, 1):
        context_lines.append(
            f"Tweet {i} (similarity={r['score']:.3f}, {r['distance_km']} km from {city_name}):\n"
            f"  Text: \"{r['text']}\"\n"
            f"  Author: @{r['author']} ({r['author_followers']} followers)\n"
            f"  Location: {r['location']}, {r['country']}\n"
            f"  Hashtags: {', '.join(r['hashtags'])}\n"
            f"  Engagement: {r['likes']} likes, {r['retweets']} retweets"
        )
    context = "\n\n".join(context_lines)

    response = client.models.generate_content(
        model=LLM_MODEL,
        contents=f"Context:\n{context}\n\nQuestion: {question}",
        config=genai.types.GenerateContentConfig(
            system_instruction=SYSTEM_PROMPT,
            temperature=0.2,
        ),
    )
    return response.text


print("✅ Geo-augmented GraphRAG ready.")

In [ ]:
# ── Geo-scoped questions across multiple cities ────────────────────
geo_rag_demos = [
    ("What are people saying about AI and machine learning?", "London, UK", 100),
    ("What do people think about cloud computing and serverless?", "San Francisco, CA", 80),
]

for question, city, radius in geo_rag_demos:
    print("=" * 70)
    print(f"❓ {question}")
    print(f"📍 Within {radius} km of {city}")
    print("=" * 70)
    
    try:
        print(geo_graph_rag_answer(question, city, radius_km=radius))
    except Exception as e:
        print(f"⚠️ Error: {e}")
    
    # Delay to avoid rate limits
    if geo_rag_demos.index((question, city, radius)) < len(geo_rag_demos) - 1:
        print("\n⏳ Waiting 20 seconds to avoid rate limits...")
        time.sleep(20)
    
    print()

---
## 🧹 Cleanup

Close the Neo4j driver when finished.

In [ ]:
driver.close()
print("✅ Neo4j connection closed.")

---
## 🏁 Conclusion

We built a complete **GraphRAG** pipeline from the social media knowledge graph:

| Component | What it does |
|-----------|-------------|
| **Vector Embeddings** | Encode tweet text into 3072-d vectors via Gemini |
| **Neo4j Vector Index** | Fast cosine-similarity search over tweet embeddings |
| **Graph Traversal** | Enrich results with author, location, hashtags, and related tweets |
| **LLM Generation** | Produce grounded answers from the enriched context |
| **Cypher Generation** | Let the LLM write graph queries for analytical questions |
| **Geospatial Queries** | Filter tweets by proximity using `point.distance()` |
| **Geo-Augmented GraphRAG** | Combine vector search + geo-filter + graph traversal |

**Key Insight:** GraphRAG produces *richer, more accurate* answers than plain vector search because it leverages the **structural relationships** between entities — not just text similarity. Adding geospatial filtering lets you scope answers to a specific region.

**🚀 API Usage (Tutorial-Optimized):**
- Test queries enabled in Steps 5 & 6 (single queries to demonstrate functionality)
- Step 8: 2 comparison questions (traditional RAG vs GraphRAG)
- Step 10: 2 analytical questions (Cypher-augmented generation)
- Step 11: 2 geospatial demos + 2 geo-augmented GraphRAG queries
- Rate-limiting delays (20s between multi-call operations) to avoid quota issues
- Balanced between comprehensive demonstration and API cost efficiency

**Next Steps:**
- Add **community detection** to find clusters of related users
- Implement **hybrid search** (keyword + vector + graph)

- Build a **Streamlit app** for interactive graph exploration- Add **temporal analysis** — how do topics evolve over time?